In [1]:
import pandas as pd
import numpy as np

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [2]:
listings_original = pd.read_csv('./Resources/listings.csv')
cols = listings_original.columns.tolist()
col_dict = {}
for i in range(len(cols)):
    col_dict[i] = cols[i]

In [9]:
# fetch all columns that are related to listings and create DF
listing_cols = [
    0,4,16,34,35,40,48,49,50,51,52,53,54,55,57,58,59,
    60,61,62,63,64,65,68,69,70,71,73,76,77,78,79,80,
    81,82,86,87,88,89,91
]
listings_df = listings_original.iloc[:, listing_cols]
listings_df.head(3)

,id,name,host_id,street,neighbourhood,zipcode,property_type,room_type,accommodates,bathrooms,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,reviews_per_month
0,241032,Stylish Queen Anne Apartment,956883,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,98119,Apartment,Entire home/apt,4,1.0,...,10.0,10.0,10.0,9.0,10.0,f,moderate,f,f,4.07
1,953595,Bright & Airy Queen Anne Apartment,5177328,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,98119,Apartment,Entire home/apt,4,1.0,...,10.0,10.0,10.0,10.0,10.0,f,strict,t,t,1.48
2,3308979,New Modern House-Amazing water view,16708587,"West Lee Street, Seattle, WA 98119, United States",Queen Anne,98119,House,Entire home/apt,11,4.5,...,10.0,10.0,10.0,10.0,10.0,f,strict,f,f,1.15


In [10]:
# check to ensure that all listings are unique
print("length: " + str(len(listings_df['id'])))
len(listings_df['id']) == len(listings_df['id'].unique())

length: 3818


True

In [11]:
listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 40 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   name                              3818 non-null   object 
 2   host_id                           3818 non-null   int64  
 3   street                            3818 non-null   object 
 4   neighbourhood                     3402 non-null   object 
 5   zipcode                           3811 non-null   object 
 6   property_type                     3817 non-null   object 
 7   room_type                         3818 non-null   object 
 8   accommodates                      3818 non-null   int64  
 9   bathrooms                         3802 non-null   float64
 10  bedrooms                          3812 non-null   float64
 11  beds                              3817 non-null   float64
 12  bed_ty

In [12]:
# split out review-related cols to make listings table less huge
# also allow simpler queries for review-related joins
base_cols = np.concatenate([np.arange(0,27), np.arange(35,39)])
review_cols = np.concatenate([[0], np.arange(27,35), [39]])
base_listings_df = listings_df.iloc[:, base_cols]
listing_reviews_df = listings_df.iloc[:, review_cols]
base_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   name                              3818 non-null   object 
 2   host_id                           3818 non-null   int64  
 3   street                            3818 non-null   object 
 4   neighbourhood                     3402 non-null   object 
 5   zipcode                           3811 non-null   object 
 6   property_type                     3817 non-null   object 
 7   room_type                         3818 non-null   object 
 8   accommodates                      3818 non-null   int64  
 9   bathrooms                         3802 non-null   float64
 10  bedrooms                          3812 non-null   float64
 11  beds                              3817 non-null   float64
 12  bed_ty

In [13]:
# retype availabilities as percentages
base_listings_df['availability_30'] = base_listings_df['availability_30'].astype('int') / 30
base_listings_df['availability_60'] = base_listings_df['availability_60'].astype('int') / 60
base_listings_df['availability_90'] = base_listings_df['availability_90'].astype('int') / 90
base_listings_df['availability_365'] = base_listings_df['availability_365'].astype('int') / 365

# create new, single column with avg availability and remove old columns
base_listings_df['avg_availability'] = (base_listings_df['availability_30'] + base_listings_df['availability_60'] + 
                                       base_listings_df['availability_90'] + base_listings_df['availability_365']) / 4
base_listings_df.drop(['availability_30','availability_60','availability_90','availability_365'], inplace=True, axis=1)
base_listings_df.sort_values('avg_availability', ascending=False)['avg_availability'].head(3)

921     1.0
2648    1.0
604     1.0
Name: avg_availability, dtype: float64

In [14]:
# retype number columns (bathrooms can be half)
base_listings_df['accommodates'] = base_listings_df['accommodates'].astype('int')
base_listings_df['bathrooms'].fillna('-1.0', inplace=True)
base_listings_df['bathrooms'] = base_listings_df['bathrooms'].replace('', '-1.0').astype('float')
base_listings_df['guests_included'] = base_listings_df['guests_included'].astype('int')
base_listings_df['minimum_nights'] = base_listings_df['minimum_nights'].astype('int')
base_listings_df['maximum_nights'] = base_listings_df['maximum_nights'].astype('int')

# extremely few rows missing data; okay to drop
base_listings_df['bedrooms'].fillna('-1', inplace=True)
base_listings_df['bedrooms'] = base_listings_df['bedrooms'].replace('', '-1').astype('int')
base_listings_df['beds'].fillna('-1', inplace=True)
base_listings_df['beds'] = base_listings_df['beds'].replace('', '-1').astype('int')

# zipcode has one weird outlier with a newline; bad entry maybe
base_listings_df['zipcode'].fillna('-1', inplace=True)
base_listings_df['zipcode'] = base_listings_df['zipcode'].replace('[0-9]+\n','', regex=True).replace('', '-1').astype('int')

# method for cleaning object-type columns with money values
def cleanPrice(column):
    curr_str = column.str
    if curr_str == '':
        return curr_str.replace('', '-1.0').astype('float')
    return curr_str.replace('$','').str.replace(',','').astype('float')

# clean price columns
base_listings_df['price'] = cleanPrice(base_listings_df['price'])

# all other money values have some null / "N/A" rows we need to clean first
base_listings_df['weekly_price'] = cleanPrice(base_listings_df['weekly_price'])
base_listings_df['monthly_price'] = cleanPrice(base_listings_df['monthly_price'])
base_listings_df['security_deposit'] = cleanPrice(base_listings_df['security_deposit'])
base_listings_df['cleaning_fee'] = cleanPrice(base_listings_df['cleaning_fee'])
base_listings_df['extra_people'] = cleanPrice(base_listings_df['extra_people'])

base_listings_df['weekly_price'].fillna(-1.0, inplace=True)
base_listings_df['monthly_price'].fillna(-1.0, inplace=True)
base_listings_df['security_deposit'].fillna(-1.0, inplace=True)
base_listings_df['cleaning_fee'].fillna(-1.0, inplace=True)
base_listings_df['extra_people'].fillna(-1.0, inplace=True)

# clean to-be boolean columns
base_listings_df['require_guest_profile_picture'] = base_listings_df['require_guest_profile_picture'].str.replace('t','True')
base_listings_df['require_guest_profile_picture'] = base_listings_df['require_guest_profile_picture'].str.replace('f','False').astype('bool')
base_listings_df['require_guest_phone_verification'] = base_listings_df['require_guest_phone_verification'].str.replace('t','True')
base_listings_df['require_guest_phone_verification'] = base_listings_df['require_guest_phone_verification'].str.replace('f','False').astype('bool')
base_listings_df['instant_bookable'] = base_listings_df['instant_bookable'].str.replace('t','True')
base_listings_df['instant_bookable'] = base_listings_df['instant_bookable'].str.replace('f','False').astype('bool')

# all remaining columns to strings
base_listings_df['name'] = base_listings_df['name'].astype('string')
base_listings_df['street'] = base_listings_df['street'].astype('string')
base_listings_df['neighbourhood'] = base_listings_df['neighbourhood'].astype('string')
base_listings_df['property_type'] = base_listings_df['property_type'].astype('string')
base_listings_df['room_type'] = base_listings_df['room_type'].astype('string')
base_listings_df['bed_type'] = base_listings_df['bed_type'].astype('string')
base_listings_df['cancellation_policy'] = base_listings_df['cancellation_policy'].astype('string')

In [15]:
# lambda example found in: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
base_listings_df['amenities'] = base_listings_df['amenities'].apply(lambda amenities: str(amenities.count(',') + 1))
base_listings_df['amenities'] = base_listings_df['amenities'].astype('int')
base_listings_df = base_listings_df.rename(columns={ 'amenities': 'amenities_count' })

In [16]:
# review-related column cleaning; similar to above for non-review cols
listing_reviews_df['review_scores_rating'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_rating'] = listing_reviews_df['review_scores_rating'].astype('int')
listing_reviews_df['review_scores_accuracy'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_accuracy'] = listing_reviews_df['review_scores_accuracy'].astype('int')
listing_reviews_df['review_scores_cleanliness'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_cleanliness'] = listing_reviews_df['review_scores_cleanliness'].astype('int')
listing_reviews_df['review_scores_checkin'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_checkin'] = listing_reviews_df['review_scores_checkin'].astype('int')
listing_reviews_df['review_scores_communication'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_communication'] = listing_reviews_df['review_scores_communication'].astype('int')
listing_reviews_df['review_scores_location'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_location'] = listing_reviews_df['review_scores_location'].astype('int')
listing_reviews_df['review_scores_value'].fillna(-1, inplace=True)
listing_reviews_df['review_scores_value'] = listing_reviews_df['review_scores_value'].astype('int')
listing_reviews_df['reviews_per_month'].fillna(-1.0, inplace=True)
listing_reviews_df = listing_reviews_df.rename(columns={'id': 'listing_id'})
listing_reviews_df.index.names = ['score_id']
listing_reviews_df.head(3)

,listing_id,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
score_id,,,,,,,,,,
0,241032,207,95,10,10,10,10,9,10,4.07
1,953595,43,96,10,10,10,10,10,10,1.48
2,3308979,20,97,10,10,10,10,10,10,1.15


In [17]:
# save listings-related tables as new CSVs
listing_reviews_df.to_csv('./Resources/listing_scores.csv')
base_listings_df.to_csv('./Resources/listings_cleaned.csv', index=False)

In [18]:
# final check on exported column types
base_listings_df.info()
# listing_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   name                              3818 non-null   string 
 2   host_id                           3818 non-null   int64  
 3   street                            3818 non-null   string 
 4   neighbourhood                     3402 non-null   string 
 5   zipcode                           3818 non-null   int32  
 6   property_type                     3817 non-null   string 
 7   room_type                         3818 non-null   string 
 8   accommodates                      3818 non-null   int32  
 9   bathrooms                         3818 non-null   float64
 10  bedrooms                          3818 non-null   int32  
 11  beds                              3818 non-null   int32  
 12  bed_ty